In [1]:
import numpy as np
import librosa


# get the mfcc of wav form
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)# res_type='kaiser_fast')
    print(wave, sr)
    wave = wave[::3]
    mfcc = np.mean(librosa.feature.mfcc(wave, sr=16000, n_mfcc = 40).T, axis=0)
    mfcc = np.pad(mfcc, pad_width=(0, 0), mode='constant')
    return mfcc

In [2]:
import numpy as np
import os
from keras import utils as np_utils

DATA_PATH = "data_new/"

# function to get the labels
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, np_utils.to_categorical(label_indices)

Using TensorFlow backend.


In [3]:
import numpy as np


# saving data to an numpy array
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)
#save_data_to_array()

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [5]:
from keras.preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 4

# Save data to array file first
save_data_to_array(max_pad_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 10
channel = 1
epochs = 800
batch_size = 100
verbose = 1
num_classes = 5

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

[0.         0.         0.         ... 0.0007019  0.00213623 0.00366211] 48000
[0.00915527 0.01034546 0.0112915  ... 0.00314331 0.0027771  0.00244141] 48000
[-0.0022583  -0.00436401 -0.0065918  ... -0.00134277 -0.00131226
 -0.00109863] 48000
[-0.01242065 -0.01422119 -0.01480103 ...  0.00454712  0.00479126
  0.00482178] 48000
[ 0.00839233  0.00741577  0.006073   ... -0.00024414 -0.00048828
 -0.00073242] 48000
[-0.00979614 -0.00897217 -0.00747681 ... -0.0027771  -0.00286865
 -0.00265503] 48000
[ 0.          0.          0.         ... -0.00064087 -0.00021362
 -0.00054932] 48000
[0.         0.         0.         ... 0.00491333 0.00376892 0.00260925] 48000
[ 0.          0.          0.         ... -0.00770569 -0.00752258
 -0.00801086] 48000
[ 0.          0.          0.         ... -0.04862976 -0.04605103
 -0.04412842] 48000
[ 0.          0.          0.         ... -0.01068115 -0.01121521
 -0.01185608] 48000
[ 0.          0.          0.         ... -0.02433777 -0.02113342
 -0.01713562] 48000
[

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(10, 4, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Instructions for updating:
Use tf.cast instead.
Train on 25 samples, validate on 18 samples
Epoch 1/800
25/25 [==============================] - 2s 73ms/step - loss: 11.0243 - acc: 0.1600 - val_loss: 7.9309 - val_acc: 0.3889
Epoch 2/800
25/25 [==============================] - 0s 91us/step - loss: 9.5719 - acc: 0.3200 - val_loss: 5.4134 - val_acc: 0.4444
Epoch 3/800
25/25 [==============================] - 0s 85us/step - loss: 8.1837 - acc: 0.3600 - val_loss: 5.9440 - val_acc: 0.6111
Epoch 4/800
25/25 [==============================] - 0s 84us/step - loss: 8.9091 - acc: 0.2400 - val_loss: 6.8117 - val_acc: 0.3889
Epoch 5/800
25/25 [==============================] - 0s 141us/step - loss: 7.3524 - acc: 0.4000 - val_loss: 6.3736 - val_acc: 0.6111
Epoch 6/800
25/25 [==============================] - 0s 209us/step - loss: 8.2190 - acc: 0.3600 - val_loss: 7.8296 - val_acc: 0.2222
Epoch 7/800
25/25 [==============================] - 0s 87us/step - loss: 8.2464 - acc: 0.3600 - val_loss: 5.9774

Epoch 60/800
25/25 [==============================] - 0s 87us/step - loss: 5.0097 - acc: 0.6000 - val_loss: 0.0870 - val_acc: 0.9444
Epoch 61/800
25/25 [==============================] - 0s 90us/step - loss: 3.3374 - acc: 0.5600 - val_loss: 0.6275 - val_acc: 0.8333
Epoch 62/800
25/25 [==============================] - 0s 75us/step - loss: 1.9466 - acc: 0.7600 - val_loss: 0.1255 - val_acc: 0.9444
Epoch 63/800
25/25 [==============================] - 0s 74us/step - loss: 3.7163 - acc: 0.5600 - val_loss: 0.0051 - val_acc: 1.0000
Epoch 64/800
25/25 [==============================] - 0s 77us/step - loss: 3.7859 - acc: 0.5600 - val_loss: 0.1839 - val_acc: 0.8333
Epoch 65/800
25/25 [==============================] - 0s 82us/step - loss: 3.8355 - acc: 0.6800 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 66/800
25/25 [==============================] - 0s 113us/step - loss: 3.1691 - acc: 0.5600 - val_loss: 0.7352 - val_acc: 0.7778
Epoch 67/800
25/25 [==============================] - 0s 152us/step 

Epoch 121/800
25/25 [==============================] - 0s 117us/step - loss: 0.8382 - acc: 0.8000 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 122/800
25/25 [==============================] - 0s 273us/step - loss: 0.4030 - acc: 0.8800 - val_loss: 0.0083 - val_acc: 1.0000
Epoch 123/800
25/25 [==============================] - 0s 224us/step - loss: 0.1273 - acc: 0.9600 - val_loss: 0.0118 - val_acc: 1.0000
Epoch 124/800
25/25 [==============================] - 0s 143us/step - loss: 1.2110 - acc: 0.8400 - val_loss: 0.0157 - val_acc: 1.0000
Epoch 125/800
25/25 [==============================] - 0s 181us/step - loss: 0.9645 - acc: 0.8000 - val_loss: 0.0090 - val_acc: 1.0000
Epoch 126/800
25/25 [==============================] - 0s 85us/step - loss: 0.3632 - acc: 0.9600 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 127/800
25/25 [==============================] - 0s 151us/step - loss: 0.4067 - acc: 0.9600 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 128/800
25/25 [==============================] - 0

Epoch 182/800
25/25 [==============================] - 0s 153us/step - loss: 0.1812 - acc: 0.9600 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 183/800
25/25 [==============================] - 0s 130us/step - loss: 0.8397 - acc: 0.8800 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 184/800
25/25 [==============================] - 0s 74us/step - loss: 0.3641 - acc: 0.8400 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 185/800
25/25 [==============================] - 0s 81us/step - loss: 0.3437 - acc: 0.9200 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 186/800
25/25 [==============================] - 0s 73us/step - loss: 0.2976 - acc: 0.9600 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 187/800
25/25 [==============================] - 0s 105us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 188/800
25/25 [==============================] - 0s 153us/step - loss: 0.8875 - acc: 0.8800 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 189/800
25/25 [==============================] - 0s 

Epoch 243/800
25/25 [==============================] - 0s 73us/step - loss: 0.5000 - acc: 0.9600 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 244/800
25/25 [==============================] - 0s 68us/step - loss: 0.0268 - acc: 1.0000 - val_loss: 0.0055 - val_acc: 1.0000
Epoch 245/800
25/25 [==============================] - 0s 71us/step - loss: 0.0298 - acc: 1.0000 - val_loss: 0.0031 - val_acc: 1.0000
Epoch 246/800
25/25 [==============================] - 0s 68us/step - loss: 0.0938 - acc: 0.9600 - val_loss: 7.2383e-04 - val_acc: 1.0000
Epoch 247/800
25/25 [==============================] - 0s 72us/step - loss: 0.0968 - acc: 0.9600 - val_loss: 5.9563e-04 - val_acc: 1.0000
Epoch 248/800
25/25 [==============================] - 0s 77us/step - loss: 0.0111 - acc: 1.0000 - val_loss: 5.8323e-04 - val_acc: 1.0000
Epoch 249/800
25/25 [==============================] - 0s 189us/step - loss: 0.2904 - acc: 0.9200 - val_loss: 4.1736e-04 - val_acc: 1.0000
Epoch 250/800
25/25 [========================

...

Epoch 541/800
25/25 [==============================] - 0s 78us/step - loss: 2.2506e-04 - acc: 1.0000 - val_loss: 9.9686e-05 - val_acc: 1.0000
Epoch 542/800
25/25 [==============================] - 0s 93us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 9.8284e-05 - val_acc: 1.0000
Epoch 543/800
25/25 [==============================] - 0s 81us/step - loss: 2.7235e-04 - acc: 1.0000 - val_loss: 9.8436e-05 - val_acc: 1.0000
Epoch 544/800
25/25 [==============================] - 0s 96us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 1.0990e-04 - val_acc: 1.0000
Epoch 545/800
25/25 [==============================] - 0s 177us/step - loss: 0.0041 - acc: 1.0000 - val_loss: 1.0106e-04 - val_acc: 1.0000
Epoch 546/800
25/25 [==============================] - 0s 81us/step - loss: 2.3218e-04 - acc: 1.0000 - val_loss: 1.0080e-04 - val_acc: 1.0000
Epoch 547/800
25/25 [==============================] - 0s 73us/step - loss: 0.0020 - acc: 1.0000 - val_loss: 1.0095e-04 - val_acc: 1.0000
Epoch 548/800
25/25 [

Epoch 600/800
25/25 [==============================] - 0s 157us/step - loss: 6.3913e-04 - acc: 1.0000 - val_loss: 4.4183e-04 - val_acc: 1.0000
Epoch 601/800
25/25 [==============================] - 0s 225us/step - loss: 0.0288 - acc: 1.0000 - val_loss: 3.9506e-04 - val_acc: 1.0000
Epoch 602/800
25/25 [==============================] - 0s 197us/step - loss: 0.0727 - acc: 0.9600 - val_loss: 4.8064e-04 - val_acc: 1.0000
Epoch 603/800
25/25 [==============================] - 0s 132us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 5.2786e-04 - val_acc: 1.0000
Epoch 604/800
25/25 [==============================] - 0s 166us/step - loss: 4.0974e-04 - acc: 1.0000 - val_loss: 5.2849e-04 - val_acc: 1.0000
Epoch 605/800
25/25 [==============================] - 0s 187us/step - loss: 1.1838e-04 - acc: 1.0000 - val_loss: 5.2926e-04 - val_acc: 1.0000
Epoch 606/800
25/25 [==============================] - 0s 188us/step - loss: 7.2654e-05 - acc: 1.0000 - val_loss: 5.2778e-04 - val_acc: 1.0000
Epoch 607/8

Epoch 659/800
25/25 [==============================] - 0s 99us/step - loss: 0.0083 - acc: 1.0000 - val_loss: 2.0014e-04 - val_acc: 1.0000
Epoch 660/800
25/25 [==============================] - 0s 323us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 2.0218e-04 - val_acc: 1.0000
Epoch 661/800
25/25 [==============================] - 0s 94us/step - loss: 3.4617e-04 - acc: 1.0000 - val_loss: 2.0317e-04 - val_acc: 1.0000
Epoch 662/800
25/25 [==============================] - 0s 126us/step - loss: 2.0969e-05 - acc: 1.0000 - val_loss: 2.0312e-04 - val_acc: 1.0000
Epoch 663/800
25/25 [==============================] - 0s 68us/step - loss: 0.0221 - acc: 1.0000 - val_loss: 2.0918e-04 - val_acc: 1.0000
Epoch 664/800
25/25 [==============================] - 0s 144us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 2.0975e-04 - val_acc: 1.0000
Epoch 665/800
25/25 [==============================] - 0s 224us/step - loss: 0.0079 - acc: 1.0000 - val_loss: 1.7537e-04 - val_acc: 1.0000
Epoch 666/800
25/25 [=

Epoch 718/800
25/25 [==============================] - 0s 252us/step - loss: 0.0103 - acc: 1.0000 - val_loss: 4.7036e-04 - val_acc: 1.0000
Epoch 719/800
25/25 [==============================] - 0s 120us/step - loss: 5.6176e-04 - acc: 1.0000 - val_loss: 4.6652e-04 - val_acc: 1.0000
Epoch 720/800
25/25 [==============================] - 0s 172us/step - loss: 0.0074 - acc: 1.0000 - val_loss: 4.7027e-04 - val_acc: 1.0000
Epoch 721/800
25/25 [==============================] - 0s 143us/step - loss: 1.3332e-04 - acc: 1.0000 - val_loss: 4.6655e-04 - val_acc: 1.0000
Epoch 722/800
25/25 [==============================] - 0s 213us/step - loss: 8.7461e-04 - acc: 1.0000 - val_loss: 4.4387e-04 - val_acc: 1.0000
Epoch 723/800
25/25 [==============================] - 0s 161us/step - loss: 0.0834 - acc: 0.9600 - val_loss: 4.7967e-04 - val_acc: 1.0000
Epoch 724/800
25/25 [==============================] - 0s 244us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 4.7793e-04 - val_acc: 1.0000
Epoch 725/800
2

Epoch 777/800
25/25 [==============================] - 0s 97us/step - loss: 1.9765e-04 - acc: 1.0000 - val_loss: 1.2866e-04 - val_acc: 1.0000
Epoch 778/800
25/25 [==============================] - 0s 85us/step - loss: 3.2124e-05 - acc: 1.0000 - val_loss: 1.2872e-04 - val_acc: 1.0000
Epoch 779/800
25/25 [==============================] - 0s 215us/step - loss: 8.2420e-05 - acc: 1.0000 - val_loss: 1.2881e-04 - val_acc: 1.0000
Epoch 780/800
25/25 [==============================] - 0s 82us/step - loss: 0.0130 - acc: 1.0000 - val_loss: 1.1918e-04 - val_acc: 1.0000
Epoch 781/800
25/25 [==============================] - 0s 91us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 1.1556e-04 - val_acc: 1.0000
Epoch 782/800
25/25 [==============================] - 0s 90us/step - loss: 0.0191 - acc: 1.0000 - val_loss: 1.4141e-04 - val_acc: 1.0000
Epoch 783/800
25/25 [==============================] - 0s 331us/step - loss: 0.0157 - acc: 1.0000 - val_loss: 2.3364e-04 - val_acc: 1.0000
Epoch 784/800
25/25 

In [8]:
import pyaudio
import wave
import subprocess
import os
import time
import threading


class Recorder():
    #Defines sound properties like frequency and channels
    def __init__(self, chunk=1024, channels=2, rate=44100):
        self.CHUNK = chunk
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = channels
        self.RATE = rate
        self._running = True
        self._frames = []

    #Start recording sound
    def start(self):
        threading._start_new_thread(self.__recording, ())

    def __recording(self):
        #Set running to True and reset previously recorded frames
        self._running = True
        self._frames = []
        #Create pyaudio instance
        p = pyaudio.PyAudio()
        #Open stream
        stream = p.open(format=self.FORMAT,
                        channels=self.CHANNELS,
                        rate=self.RATE,
                        input=True,
                        frames_per_buffer=self.CHUNK)
        # To stop the streaming, new thread has to set self._running to false
        # append frames array while recording
        while(self._running):
            data = stream.read(self.CHUNK)
            self._frames.append(data)

        # Interrupted, stop stream and close it. Terinate pyaudio process.
        stream.stop_stream()
        stream.close()
        p.terminate()

    # Sets boolean to false. New thread needed.
    def stop(self):
        self._running = False

    #Save file to filename location as a wavefront file.
    def save(self, filename):
        print("Saving")
        p = pyaudio.PyAudio()
        if not filename.endswith(".wav"):
            filename = filename + ".wav"
        wf = wave.open(filename, 'wb')
        wf.setnchannels(self.CHANNELS)
        wf.setsampwidth(p.get_sample_size(self.FORMAT))
        wf.setframerate(self.RATE)
        wf.writeframes(b''.join(self._frames))
        wf.close()
        print("Saved")





In [9]:
def getAudio():
    rec = Recorder()
    print("Start recording")
    rec.start()
    time.sleep(5)
    print("Stop recording")
    rec.stop()
    print("Saving")
    rec.save("test.wav")

In [10]:
def makePred():
    print(predict('test.wav', model=model))

In [11]:
from tkinter import *
import tkinter.messagebox



window = Tk()
window.geometry("600x600")
window.title("testing")

def printt():
    first = fn.get()
    second = ln.get()
    dob1 = dob.get()
    gen = radio_var.get()
    print(f"{first}{second}{dob1}{gen}")
    tkinter.messagebox.showinfo("welcome", 'user is registered!!')

def exit1():
    exit()


def checking_window():
    win = Tk()
    win.title("welcome to the prediction")
    win.geometry("600x600")
    label5 = Label(win, text="let's check your voice prediction")
    label5.place(x=30, y=70)
    butn1 = Button(win, text='Record voice', width=12, command=getAudio)
    butn1.place(x=80, y=120)
    
    butn2 = Button(win, text='Predict', width=12, command=makePred)
    butn2.place(x=400, y=120)
    
    

fn = StringVar()
ln = StringVar()
dob = StringVar()
radio_var = StringVar()

label = Label(window, text = "registration form")
label.place(x=90, y =53)

label1 = Label(window, text = "fname:-")
label1.place(x=70, y =130)
entry1 = Entry(window, textvar = fn)
entry1.place(x=130, y=132)


label2 = Label(window, text = "lname:-")
label2.place(x=70, y =170)
entry1 = Entry(window, textvar = ln)
entry1.place(x=130, y=172)


label3 = Label(window, text = "dob:-")
label3.place(x=70, y =210)
entry3 = Entry(window, textvar = dob)
entry3.place(x=130, y=212)


label4 = Label(window, text = "gender :-")
label4.place(x=70, y=242)

r1 = Radiobutton(window, text="Male", variable=radio_var, value="Male")
r1.place(x=130, y=245)
r2 = Radiobutton(window, text="Female", variable=radio_var, value="Female")
r2.place(x=200, y=245)

b1 = Button(window, text='Signup', width=12, command=printt)
b1.place(x=150, y=440)

b2 = Button(window, text='exit', width=12, command=exit1)
b2.place(x=300, y=440)

b3 = Button(window, text='predict voice', width=12, command=checking_window)
b3.place(x=220, y=500)


window.mainloop()


Start recording
Stop recording
Saving
Saving
Saved
[-0.01596069 -0.01644897 -0.01715088 ...  0.02210999  0.02072144
  0.02307129] 44100
jeet
